# **Learning to Hack Game Weather Conditions**
#### An End-to-End Machine Learning Workflow for Formula One In-Game Weather Forecasting
#### Team Twin AI ####

***Contributors to this Notebook:*** Zion Pibowei, Temitayo Adejuyigbe, Anosike Chimaobi Nice

## Background

Formula 1 is one of the most competitive sports in the world. Engineers and technicians from every team use weather radar screens, provided by Ubimet to the teams, which allows them to track the current weather and make predictions during the race. Race engineers relay precise information to drivers, including:

- How many minutes until it starts raining
- Intensity of the rain
- Which corner will be hit first by the rain
- Duration of the rain

Points, and even races sometimes, are won and lost based on making sense of what the weather is going to do during a race, and being prepared as a team to act accordingly.

Therefore, weather forecasting takes a big part on the possible outcome of a race.

Similarly, F1 2021, the official Formula 1 videogame developed by Codemasters, uses a physics engine that behaves like the real world.

## The Challenge

In this challenge, we are required to analyse historical weather data from the RedBull Racing eSports team to build a high-performing model that is able to make accurate weather predictions/forecasts. Our objective is to predict the weather type 5, 10, 15, 30 and 60 minutes after a timestamp, and the corresponding rain percentage probability at each time. 

***Our solution is divided into 4 Sections, each constituting a workflow on its own:***

- Part I: Initial Data Analysis and Preprocessing
- Part II: EDA and Feature Selection
- Part III: Modelling Methodology
- Part IV: Predictions and Exporting

## Part I: Initial Data Analysis and Preprocessing
<h4><b>Overview</b></h4>

This is the IDA and Preprocessing component of our solution to the FormulaAI Hack 2022 Competition. The workflow for this notebook is outlined as follows: 
- Getting the Data
- First Insights: Making Sense of the Data
- Data Integrity Assessments
- Cleaning the Data


In [1]:
import os

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None, 'display.max_rows', 100, 'display.precision', 15)

import matplotlib.pyplot as plt
from matplotlib import figure
%matplotlib inline

import seaborn as sns

import random
import time
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

!conda install -c conda-forge deepchecks -y
import deepchecks as dc
from deepchecks.checks.integrity.is_single_value import IsSingleValue
from deepchecks.checks.integrity.data_duplicates import DataDuplicates
from deepchecks.checks import DataDuplicates
from deepchecks.checks.integrity import LabelAmbiguity
from deepchecks.base import Dataset, Suite

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# Reading the CSV file
data = pd.read_csv('../input/formulaaihackathon2022/weather.csv',low_memory=False)
#data = pd.read_csv('weather.csv')

<br>
<h4><b>1. First Insights: Making Sense of the Data </b></h4>

In [4]:
data.shape

In [5]:
data.info() #check the overall information about the dataset

In [6]:
data.head(2)

<br>In this dataframe, we see that the last column is an unnamed column. Before we drop it, let us inspect the proportion of NaN values it contains.<br>

In [7]:
print('Missing values:',data['Unnamed: 58'].isnull().sum())
print('Proportion of missing values: {}%'.format(data['Unnamed: 58'].isnull().sum()/data.shape[0]*100))

Clearly, the unnamed column is entirely filled with missing values and, as such, has no impact in our workflow. It's presence in the data is most likely due to encoding. Thus, the first treatment to our data is to drop this column.

In [8]:
data.drop('Unnamed: 58', axis = 1, inplace = True)
data.shape[1]

Now, let's obtain summary statistics for our data 

In [9]:
data.describe()

<br><b>At a glance:</b>
- A quick inspection of the count row shows us that a number of columns contain missing values, ranging from small to large.
- A quick inspection of the standard deviation shows us that some columns have zero variance, indicating that <b>each of these columns contain ONLY ONE distinct value</b>. Typically, variables whose standard deviations tend to zero have fewer distinct values.
- A quick inspection of the min and max shows us that the very columns having 0 standard deviation <b>contain equal values of minimum and maximum</b>, validating our claim that these columns have only one distinct value.

In this project, we will carry out exhaustive analysis of the data to address the implictations of the forgoing discoveries.</b>


<h4><b>2. Data Integrity Assessments</b></h4>
<p>In this section, we will investigate the integrity of the data and uncover any data quality issues that may be present. The insights we obtain in this section will guide us on how to resolve these issues pragmatically in the next section.</p>

<p><b>(a) Unique Values</b></p>

Our ultimate goal is to build <b>a model that learns the evolution of weather conditions over time</b>. Therefore, we are interested in columns that show variation of values over time. Columns that contain only one unique value <b>may not provide predictive power for the model</b>. We will validate this assumption when we implement feature contribution checks ahead of our model methodology.
<p> First, we make a general inspection of the number of unique values contained in all the columns.</p>

In [10]:
data.nunique(axis=0).sort_values().to_frame() #check for unique values and sort them into frames

From the above result, we can see that there are 7 columns that contain only one unique value. Below, we obtain further information about what these exact values are.

In [11]:
sv = IsSingleValue()
sv.run(data)

<br>
<p><b>(b) Data Duplicates<b></p>
We need to run a duplicate check to find if there are multiple instances of identical samples in our dataset. One reason is that duplicates could be an indicator for a problem in the data pipeline that requires attention. The other is that they can potentially increase the weight that a machine learning model gives to samples. 

In [12]:
print('Proportion of duplicates: {}%'.format(len(data[data.duplicated()])/data.shape[0]*100))
data[data.duplicated()] #check for average duplicate values in the dataset

Now, this is only partially informative. We only know that 2057230 samples, representing ~57% of the data, are duplicated. But this doesn't tell us the number of times each example of duplicate data appears. We will obtain the desired information by implementing the following additional checks.

In [13]:
from deepchecks.checks import DataDuplicates
DataDuplicates().run(data)

We can summarise this check by defining a check condition that sets the baseline of duplicate ratio as 0. This will expose any violation to the condition and reveal the present duplicate ratio.

In [14]:
check = DataDuplicates()
check.add_condition_ratio_not_greater_than(0)
result = check.run(data)
result.show(show_additional_outputs=False)

We are interested in knowing whether the duplicates observed here were intentionally intended to be part of the data. However, if this is an hidden issue that is not expected to occur, then we will need to resolve it. We will revisit this in the EDA component of our workflow.

<br>
<p><b>(c) Label Ambiguity</b></p>

We would also like to check whether there are identical samples in the data with different labels. This alerts us to further verify whether or not the data was mislabelled, as mislabelled data could confuse the model and lead to lower model performance.

In [15]:
#label_ambig = Dataset(data, label='M_WEATHER')
#LabelAmbiguity().run(label_ambig)

Again, we summarise this check by defining a check condition that sets the baseline of ambiguous sample ratio as 0. This will expose any violation to the condition and reveal the present ambiguous sample ratio.

In [16]:
#check = LabelAmbiguity()
#check.add_condition_ambiguous_sample_ratio_not_greater_than(0)
#result = check.run(label_ambig)
#result.show(show_additional_outputs=False)

Indeed, we observe that there are no identical samples with different labels.

<br>
<p><b>(d) Missing Values</b></p>

In [17]:
data.isna().sum().sort_values().to_frame() #check for missing values and sort them into frames

In [18]:
data.notna().sum().sort_values().to_frame() #check for non-missing values and sort them into frames

From the above two cells, we immediately note the following:
- There are 18 columns with missing values, out of which 7 have only 1 missing value.
- Of the 18 columns, the number of missing values found in 8 columns (i.e., 974274 each) and the number found in 2 columns (i.e., 2598054) sum up to the length of the dataframe. 

Could there be a complimentary relationship, where columns in one set are filled in rows where those of the other set are missing?
To uncover this, we isolate the columns <b>M_WEATHER_PERCENTAGE and M_ZONE_START</b> and inspect the distribution of the missing vales across them. Due to the length of the dataframe, we slice a fraction of the data and visualise the distribution of missing values.

In [19]:
xdf = data.copy()
xdf.M_RAIN_PERCENTAGE = np.where(xdf.M_RAIN_PERCENTAGE.isnull(),'1: Missing','1: Present')
xdf.M_ZONE_START = np.where(xdf.M_ZONE_START.isnull(),'2: Missing','2: Present')
xdf.M_ZONE_START.unique()

plt.figure(figsize=(20,30))
count = 0
for i in range(1,16):
    x1 = xdf[count:count+155].M_RAIN_PERCENTAGE
    x2 = xdf[count:count+155].M_ZONE_START
    index = range(count,count+155)
    plt.subplot(5,3,i)
    plt.plot(x1,index,'bo',markersize = 2,label='Rain percentage')
    plt.plot(x2,index,'ro',markersize = 2,label='Time zone start')
    plt.ylabel('Index') 
    plt.legend(loc='upper right')
    plt.grid(False)
    count+=155

From these plots, we can generalise that the missing values in one column appear in rows where the others are filled. This holds true for the other columns across the two sets. As the insights build up, we willgain better claarity on how to prepare the data to achieve overall completeness and accuracy.

<br>
<h4><b>4. Cleaning the Data</b></h4>

<b> (a) We will drop the following rows immediately </b>
1. Rows where the number of forcast samples equals 0 as they provide no prediction at time t = 0
2. Rows where the session type is unknown (0)
3. Rows where the packet received shows a session type of NaN or 0
4. Rows where the packet received is sent while the game is paused
5. Rows where the packet received shows player is both spectating and playing online (inconsistency)
6. Rows where marshal_zone_start or marshal_zone_flag is null, as these indicates gaps in the game

In [20]:
df = data.copy()
df.shape

In [21]:
df.drop(df[df['M_NUM_WEATHER_FORECAST_SAMPLES']==0].index, inplace=True)
df[df['M_NUM_WEATHER_FORECAST_SAMPLES']==0].count()

In [22]:
df.shape

In [23]:
df.drop(df[df['M_SESSION_TYPE']==0].index, inplace=True)
df.shape

In [24]:
df.drop(df[df['M_GAME_PAUSED']==1].index, inplace=True)
df.shape

In [25]:
df.drop(df[(df['M_IS_SPECTATING'] == 1) & (df['M_NETWORK_GAME'] == 1)].index, inplace=True)
df.shape

In [26]:
df.drop(df[df['M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE'].isnull()].index, inplace=True)
df.shape

This last operation leaves the zone start and zone flag columns with NaN values as we can see below. We will therefore eliminate these columns in due time.

In [27]:
df['M_ZONE_FLAG'].isnull().sum()

<br><b>(a) We will drop the following columns immediately:</b>
1. Redundant columns not included with the packet, starting with gamehost and timestamp
2. Redundant columns with single unique values (i.e., predominantly ID columns)
3. Duplicated Columns
4. Already Engineered Columns
5. Forcast samples columns outside weather and %rainfall, since this is a weather forecast project

We start by aggregating the session duration and Session time left column to generate a new column representing the time delta in the game.

In [28]:
df['M_SESSION_TIME_SPENT'] = df['M_SESSION_DURATION'] - df['M_SESSION_TIME_LEFT']

In [29]:
drop_col = ['GAMEHOST','TIMESTAMP', 'M_ZONE_FLAG', 'M_ZONE_START', 'M_SESSION_DURATION', 
           'M_SESSION_TIME_LEFT', 'M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE', 
            'M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE', 'M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE']

for col in df.columns:
    if df[col].nunique()<2:
        drop_col.append(col)
print(drop_col)
len(drop_col)

In [30]:
df.drop(drop_col, axis=1, inplace=True)

In [31]:
df.count()

In [32]:
df.isna().any().sum()

<b>(c) Filling in the 6 columns containing single missing values</b>

In [33]:
fill_col = df.columns[df.isna().any() == True]
for col in fill_col:
    df[col].fillna(df[col].mode()[0], inplace=True)
df.isna().any().sum()

In [34]:
#Shape of data before cleaning
data.shape

In [35]:
#Shape of data after cleaning
df.shape

In [36]:
#df.to_csv('./cleaned_one.csv', encoding='utf-8', index=False)

## Part II: Exploratory Data Analysis and Feature Selection
<h4><b>Overview</b></h4>

This is the EDA and Feature Egineering component of our solution to the FormulaAI Hack 2022 Competition. The workflow for this notebook is outlined as follows: 
- Multivariate Exploratory Analysis
- Feature Importance and Selection

In [37]:
import os

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None, 'display.max_rows', 100)


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


import deepchecks as dc
from deepchecks.checks.methodology import SingleFeatureContribution
from deepchecks.base import Dataset

#from sklearnex import patch_sklearn
#patch_sklearn()
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.preprocessing import scale, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats

import random
import time
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [38]:
#data= pd.read_csv('cleaned_one.csv')
#data.head()

In [39]:
#data.drop('Unnamed: 0', axis =1, inplace = True)

In [40]:
#df = data.copy()
print(df.shape)
df.isna().values.any()

<br>
<h4><b>1. Multivariate Exploratory Analysis</b></h4>
<p> Our first step is to visualise and understand the correlation that exists between the features in the data

In [41]:
corrmat = df.corr()
#f, ax = plt.subplots(figsize=(16, 12))
#sns.heatmap(corrmat, vmax=.8, annot=True, square=True, fmt='.2f')

plt.figure(figsize=(80,80))
sns.set(font_scale=2.5)
sns.heatmap(corrmat, annot=True, fmt='.1f', cmap='YlGnBu')

The above plot is too cluttered. If we zoom further in to the above plot, we find that M_WEEKEND_LINK_IDENTIFIER, M_SEASON_LINK_IDENTIFIER and M_SESSION_LINK_IDENTIFIER show
perfect correleration of 1.

In [42]:
xtremilar_col = ['M_WEEKEND_LINK_IDENTIFIER', 'M_SEASON_LINK_IDENTIFIER', 'M_SESSION_LINK_IDENTIFIER']

In [43]:
corrmat = df[xtremilar_col ].corr()
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corrmat, vmax=.4, annot=True, square=True, fmt='.1f',)

In [44]:
for col in xtremilar_col:
    print(col,'-->',df[col].nunique())

<br>For the heatmap to have indicated a correlation of 1.0, then it means one or 2 columns will most likely contain all of the values in others. We will perform a groupby on each of them to identify how many unique values in one column correspond to each unique value in another column.

In [45]:
df[xtremilar_col].groupby('M_WEEKEND_LINK_IDENTIFIER').nunique()

In [46]:
df[xtremilar_col].groupby('M_SEASON_LINK_IDENTIFIER').nunique()

In [47]:
df[xtremilar_col].groupby('M_SESSION_LINK_IDENTIFIER').nunique()

From the first result, we find that only the season link ID occurs exactly once in the weekend ID groups. However, the second result shows that the weekend ID occurs more than once in some season ID groups. Therefore, weekend ID is a superset of season ID, indicating that weekend ID captures all the information that weekend ID presents. 

Finally, we see that in the third result, both season ID and weekend ID occur exactly once in the session ID group, indicating that session ID captures all the informations that both season ID and weekend ID presents. This makes sense considering that whether the games are played in a weekend, a season, or otherwise, each game is played in session. This provides the justification for which we will be dropping both the season and weekend IDs.

In [48]:
df.drop(xtremilar_col[:-1], axis=1, inplace=True)
print(df.shape)

Now, let's zoom into the heatmap to observe the top 20 most correlated features with weather

In [49]:
#### top 20 most correlated features with Weather (correlation matrix) 

f, ax = plt.subplots(figsize=(25,20))
k = 20 #number of variables for heatmap
cols = df.corr().nlargest(k, 'M_WEATHER')['M_WEATHER'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.1f', cmap='YlGnBu', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Zooming further, we notice a correlated submatrix of 7 features which showed perfect correleration of 1. This block contains five of the seven assist features and the two dynamic racing line features. 

Our assumption is that the other two assists features that do not make it to this block, namely steering assist and braking assist, could be due to a much smaller proportion of players using these assists, as more players typically have the experience to play without these 2 supports. 

In [50]:
xtremilar_col2 = [
'M_DYNAMIC_RACING_LINE_TYPE'
,'M_DYNAMIC_RACING_LINE'
,'M_PIT_RELEASE_ASSIST'
,'M_ERSASSIST'
,'M_PIT_ASSIST'
,'M_GEARBOX_ASSIST'
,'M_STEERING_ASSIST'
,'M_BRAKING_ASSIST'
,'M_DRSASSIST']

In [51]:
corrmat2 = df[xtremilar_col2].corr()
f, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(corrmat2, vmax=1, annot=True, square=True, fmt='.1f', cmap='YlGnBu')

Now, we have added the two remaining assists and we see they are not as correlated. We begin by performing a groupby on the columns in the list, one column at a time.

In [52]:
df[xtremilar_col2].groupby('M_DYNAMIC_RACING_LINE_TYPE').nunique()

In [53]:
df[xtremilar_col2].groupby('M_DYNAMIC_RACING_LINE').nunique()

In [54]:
df[xtremilar_col2].groupby('M_ERSASSIST').nunique()

In [55]:
df[xtremilar_col2].groupby('M_PIT_ASSIST').nunique()

In [56]:
df[xtremilar_col2].groupby('M_GEARBOX_ASSIST').nunique()

In [57]:
df[xtremilar_col2].groupby('M_STEERING_ASSIST').nunique()

In [58]:
df[xtremilar_col2].groupby('M_BRAKING_ASSIST').nunique()

In [59]:
df[xtremilar_col2].groupby('M_PIT_RELEASE_ASSIST').nunique()

In [60]:
df[xtremilar_col2].groupby('M_DRSASSIST').nunique()

It's a litlle challenging deciding which of the 9 columns to retain, considering that none of the grouby results shows columns having values uniquely occuring once across the groups. An even more important question is whether we need any of these features to predict the targets. We will therefore implement a feature contribution assessment to evaluate which of the 9 (if any) have the highest predictive power score with respect to the targets.

In [61]:
_ = Dataset(df[xtremilar_col2], label=df['M_WEATHER'])
SingleFeatureContribution().run(_)

In [62]:
_ = Dataset(df[xtremilar_col2], label=df['M_RAIN_PERCENTAGE'])
SingleFeatureContribution().run(_)

We will just drop all the 9 features. Our argument is that the AI engine doesn't take these informations into consideration to make inference on expected weather condition.

In [63]:
df.drop(xtremilar_col2, axis=1, inplace=True)
print(df.shape)
df.head()

The above contribution checks showed us that we do not need any of the 9 features for our model. We will apply such an assessment on the rest of our features during feature selection. For now, we want to explore the groupings in the data.

<br><b> Exploring the groupings in the data</b>

In [64]:
de = df.copy()

In [65]:
dd = de.groupby(['M_SESSION_UID', 'M_TIME_OFFSET'])
dd.first()

In [66]:
plt.figure(figsize=(25, 20))
sns.kdeplot(
   data=df, x="M_WEATHER", hue="M_RAIN_PERCENTAGE",
   fill=True, common_norm=False, palette="coolwarm",
   alpha=.5, linewidth=0,
)
plt.grid(False)

Now, let us explore the distribution of some of the features within the groups of the data, and see how these features change over time, session to session.

In [67]:
count = 1
colour = ['blue','red','darkblue']
plt.figure(figsize=(30,200))
for col in de.columns:
    plt.subplot(14,2,count)
    dd[col].mean().plot(lw=1,color=np.random.choice(colour))
    plt.ylabel('AVERAGE_{}'.format(col)) 
    plt.title('Time Variation of {}'.format(col), fontsize = 16)
    #plt.legend(loc='center')
    plt.grid(False)
    if count<36:
        count+=1

<br>As our ultimate goal is to <b> build models that learn the evolution of in-game weather conditions over time</b> features that do not show reasonable variation over time, as revealed by the plots, will be less useful for our models.

Before dropping these features, let's transform the data by regrouping by session UID, we then sort by session time and time offset. In other words, we want the data to represent the time variation, session by session.

In [68]:
start = de.copy()
frames = []
for uid in np.sort(start['M_SESSION_UID'].unique()):
    _ = start[start['M_SESSION_UID']==uid]
    _.sort_values(by=['M_SESSION_TIME','M_TIME_OFFSET'], inplace=True,
               ascending = [True, True])
    frames.append(_)
dtime = pd.concat(frames)
dtime.shape

In [69]:
dtime.head()

In the preceeding plots, we explored the internal variations in the data grouped by session UID and time offset. Now that we have reordered our data, we will visualise the variations in the grouping by time offset alone and inspect what the seasonality looks like across features.

In [70]:
count = 1
colour = ['blue','red','darkblue']
plt.figure(figsize=(30,60))
for col in dtime.columns.unique():
    plt.subplot(7,4,count)
    dtime.groupby('M_TIME_OFFSET')[col].min().plot(lw=1,color=np.random.choice(colour))
    plt.ylabel('MIN_{}'.format(col)) 
    plt.title('Time Variation of {}'.format(col), fontsize = 16)
    #plt.legend(loc='center')
    plt.grid(False)
    if count<28:
        count+=1

<br>The distributions would have looked different from this had it been we had plotted with the mean since this would have made it more seasonal. However, max or min are more realistic in this case as they allow us detect features that have constant max or min across time offsets. However, it is not clear from this plot whether the features that show no variation across time offsets in this plot are binary.

<b>At a glance, we observe the following features are constant in time at min and max:</b><br>
- frame ID
- pit stop window ideal lap
- track id
- formula
- session type
- pit stop window latest lap
- forecast accuracy
- pit stop rejoin position
- track temperature change
- weather forecast sample
- AI difficulty
- pit speed limit
- network game

Now, we will isolate these features and study their variations ***with respect to weather*** after 5, 10, 15, 30 and 60 minutes. <b>Before this, we will perform our final row reduction by removing rows with time offset 0, as there's been no indication that these provide any predictive value</b>. This is justfied by the visualisations above which show us constant observations across this time offset value.

In [71]:
print('Shape before dropping -->', dtime.shape)
afterdrop = dtime.drop(dtime[dtime['M_TIME_OFFSET']==0.].index)
print('Shape after dropping -->', afterdrop.shape)

In [72]:
isolated = afterdrop[[
            'M_SESSION_UID',
            'M_SESSION_TYPE',
            'M_TIME_OFFSET',
            'M_FRAME_IDENTIFIER',
            'M_PIT_STOP_WINDOW_IDEAL_LAP',
            'M_TRACK_ID',
            'M_FORMULA',
            'M_PIT_STOP_WINDOW_LATEST_LAP',
            'M_PIT_STOP_REJOIN_POSITION',
            'M_TRACK_TEMPERATURE_CHANGE',
            'M_AI_DIFFICULTY',
            'M_PIT_SPEED_LIMIT',
            'M_NETWORK_GAME',
            'M_FORECAST_ACCURACY',
            'M_WEATHER_FORECAST_SAMPLES_M_WEATHER',
            'M_WEATHER'
            ]]

In [73]:
isolated = isolated[(isolated['M_TIME_OFFSET'] <= 60) & (isolated['M_TIME_OFFSET'] > 0)]
isolated['M_TIME_OFFSET'].unique()

In [74]:
count = 1
colour = ['blue','red','darkblue']
plt.figure(figsize=(30,60))
for col in isolated.columns:
    plt.subplot(7,4,count)
    isolated.groupby(['M_TIME_OFFSET','M_WEATHER'])[col].mean().plot(lw=1,color=np.random.choice(colour)) #plotting by mode, implemented by finding the highest value count
    plt.ylabel('MEAN_{}'.format(col)) 
    plt.title('Weather Variation of {}'.format(col), fontsize = 16)
    #plt.legend(loc='center')
    plt.grid(False)
    if count<16:
        count+=1

<br> We are being careful at this point not to second guess dropping of unwanted features, as many of these features, despite demonstrating no variation in the time offsets, have been shown in our earliest observations to vary session ID by session ID. We will therefore leave the rest of the preprocessing to the next section on feature selection.

In [75]:
print('Downloading data...')
afterdrop.to_csv('./eda_output.csv')
print('Done!')

<br>
<h4><b>2. Feature Selection Experimentations</b></h4>

We begin this section by obtaining a feature importance ranking. Then we perform feature selection using two algorithms side by side: first with **generic univariate select**, and second with **recursive feature elimination**. We compare the results from both procedures and immdediately eliminate those features that were not selected by both.

**(a) Ranking Feature Importance**

We will implement Kepler Variable Importance to rank feature importance for each of our targets:

In [76]:
target1 = afterdrop['M_WEATHER']
target2 = afterdrop['M_RAIN_PERCENTAGE']
features1 = afterdrop.drop('M_WEATHER', axis=1)
features2 = afterdrop.drop('M_RAIN_PERCENTAGE', axis=1)
features1.shape

In [77]:
features2.shape

<i> **Rank feature importance for Target 1** </i>🎯 

In [78]:
kepler_mutual_information = mutual_info_classif(features1, target1)

plt.subplots(1, figsize=(27, 1))
sns.heatmap(kepler_mutual_information[:, np.newaxis].T, cmap='Blues', cbar=False, linewidths=1, annot=True)
plt.yticks([], [])
plt.gca().set_xticklabels(features1.columns[0:], rotation=45, ha='right', fontsize=12)
plt.suptitle("Kepler Variable Importance (mutual_info_classif)", fontsize=18, y=1.2)
plt.gcf().subplots_adjust(wspace=0.2)
pass

<i> **Rank feature importance for Target 2** </i> 🎯 

In [79]:
kepler_mutual_information_2 = mutual_info_classif(features2, target2)

plt.subplots(1, figsize=(27, 1))
sns.heatmap(kepler_mutual_information_2[:, np.newaxis].T, cmap='Blues', cbar=False, linewidths=1, annot=True)
plt.yticks([], [])
plt.gca().set_xticklabels(features2.columns[0:], rotation=45, ha='right', fontsize=12)
plt.suptitle("Kepler Variable Importance (mutual_info_classif)", fontsize=18, y=1.2)
plt.gcf().subplots_adjust(wspace=0.2)
pass

<br> **(b) Feature Selection with Kepler Variable Importance (Generic Univariate Select)**

In [80]:
def feature_selection_table(selected, original_features):
    """
    Display a table that indicates whether or not a feature was selected
    """
    status = []
    index = []
    for col in original_features:
        if col in selected:
            index.append(col)
            status.append('Selected')
        else:
            index.append(col)
            status.append('Not Selected')
            
    _selection_table = pd.DataFrame(status, columns = ['Selection Status'], index = original_features)
    
    print('Number of features selected:', len(selected))
    print('Number of features left out:', len([col for col in original_features if col not in selected]))
    
    return _selection_table

<i> **Select features for target 1** </i>

In [81]:
#GenericUnivariateSelect(score_func=lambda X, y: X.mean(axis=0), mode='percentile', param=50)
trans = GenericUnivariateSelect(score_func=mutual_info_classif, mode = 'percentile', param=50)
kepler_X_trans = trans.fit_transform(features1, target1)

In [82]:
columns_retained_Select = features1.iloc[:, :].columns[trans.get_support()].values
features_selected_1 = pd.DataFrame(kepler_X_trans, columns=columns_retained_Select)
features_selected_1.shape

In [83]:
Kepler_Target_1 = features_selected_1.columns
feature_selection_table(Kepler_Target_1, features1.columns)

<i> **Select features for target 2** </i>

In [84]:
#GenericUnivariateSelect(score_func=lambda X, y: X.mean(axis=0), mode='percentile', param=50)
trans_2 = GenericUnivariateSelect(score_func=mutual_info_classif, mode = 'percentile', param=50)
kepler_X_trans_2 = trans_2.fit_transform(features2, target2)

In [85]:
columns_retained_Select_2 = features2.iloc[:, :].columns[trans.get_support()].values
features_selected_2 = pd.DataFrame(kepler_X_trans_2, columns=columns_retained_Select_2)
features_selected_2.shape

In [86]:
Kepler_Target_2 = features_selected_2.columns
feature_selection_table(Kepler_Target_2, features2.columns)

<br>**(c) Feature Selection by Recursive Feature Elimination (RFE)**

<i> **Eliminate features for target 1** </i>

Since target 1 (weather) is a binary class label, we will implement RFE using the logistic regression algorithm

In [87]:
estimator_1 = LogisticRegression(solver='liblinear')
rfe_1 = RFE(estimator_1, n_features_to_select=13, step=1)
fit_1 = rfe_1.fit(features1, target1)
print("Num Features: %s" % (fit_1.n_features_))
print("Selected Features: %s" % (fit_1.support_))
print("Feature Ranking: %s" % (fit_1.ranking_))

#RFE_Target_2 = fit.get_feature_names_out(features2.columns)
#RFE_Target_2

Now, we will use the following helper functions to generate an array of the names of features that were selected corresponding to the selection index position. This array will then be used to generate the dataframe that displays selected features.

In [88]:
def _check_feature_names_in(estimator, input_features=None):
    """Get output feature names for transformation.
    Parameters
    ----------
    input_features : array-like of str or None, default=None
        Input features.
        - If `input_features` is `None`, then `feature_names_in_` is
            used as feature names in. If `feature_names_in_` is not defined,
            then names are generated: `[x0, x1, ..., x(n_features_in_)]`.
        - If `input_features` is an array-like, then `input_features` must
            match `feature_names_in_` if `feature_names_in_` is defined.
    Returns
    -------
    feature_names_in : ndarray of str
        Feature names in.
    """

    feature_names_in_ = getattr(estimator, "feature_names_in_", None)
    n_features_in_ = getattr(estimator, "n_features_in_", None)

    if input_features is not None:
        input_features = np.asarray(input_features, dtype=object)
        if feature_names_in_ is not None and not np.array_equal(
            feature_names_in_, input_features
        ):
            raise ValueError("input_features is not equal to feature_names_in_")

        if n_features_in_ is not None and len(input_features) != n_features_in_:
            raise ValueError(
                "input_features should have length equal to number of "
                f"features ({n_features_in_}), got {len(input_features)}"
            )
        return input_features

    if feature_names_in_ is not None:
        return feature_names_in_

    # Generates feature names if `n_features_in_` is defined
    if n_features_in_ is None:
        raise ValueError("Unable to generate feature names without n_features_in_")

    return np.asarray([f"x{i}" for i in range(n_features_in_)], dtype=object)


def get_feature_names_out(estimator, input_features=None):
    """Mask feature names according to selected features.
    Parameters
    ----------
    input_features : array-like of str or None, default=None
        Input features.
        - If `input_features` is `None`, then `feature_names_in_` is
          used as feature names in. If `feature_names_in_` is not defined,
          then names are generated: `[x0, x1, ..., x(n_features_in_)]`.
        - If `input_features` is an array-like, then `input_features` must
          match `feature_names_in_` if `feature_names_in_` is defined.
    Returns
    -------
    feature_names_out : ndarray of str objects
        Transformed feature names.
    """
    input_features = _check_feature_names_in(estimator, input_features)
    return input_features[estimator.get_support()]

In [89]:
RFE_Target_1 = get_feature_names_out(fit_1, features1.columns)
RFE_Target_1 

In [90]:
feature_selection_table(RFE_Target_1, features1.columns)

<br><i> **Eliminate features for target 2** </i>

Since target 2 (percentage rain) is a continuous target, we will implement RFE using a decision tree regressor

In [91]:
estimator_2 = DecisionTreeRegressor()
rfe = RFE(estimator_2, n_features_to_select=13, step=1)
fit = rfe.fit(features2, target2)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

#RFE_Target_2 = fit.get_feature_names_out(features2.columns)
#RFE_Target_2

In [92]:
RFE_Target_2 = get_feature_names_out(fit, features2.columns)
RFE_Target_2

In [93]:
feature_selection_table(RFE_Target_2, features2.columns)

**Now we will create a function that combines the results from RFE and those from Kepler's, which will allow us compare between both**

In [94]:
cols_classif = ['Kepler Target 1 Results', 'RFE Target 1 Results']
results_classif = [Kepler_Target_1, RFE_Target_1]
features_classif = [features1.columns, features1.columns]

cols_reg = ['Kepler Target 2 Results', 'RFE Target 2 Results']
results_reg = [Kepler_Target_2, RFE_Target_2]
features_reg = [features2.columns, features2.columns]

In [95]:
def new_selection_table(selection_lists, features_list, cols):
    """
    Display a table combining all the feature selection results for the two targets
    """
    #cols = []
    #for j in range(len(selection_lists)):
    #    cols.append([i for i, a in locals().items() if a == '{}'.format(selection_lists[j])])
    
    tables_list = [feature_selection_table(i,j) for i,j in zip(selection_lists, features_list)]
    _selection_table = pd.DataFrame(columns = cols, index = tables_list[0].index)
    
    for table, algorithm in zip(tables_list, cols):
        _selection_table[algorithm] = table
    
    return _selection_table

In [96]:
selection_results_classif = new_selection_table(results_classif, features_classif, cols_classif)
selection_results_classif

In [97]:
selection_results_reg = new_selection_table(results_reg, features_reg, cols_reg)
selection_results_reg

We can see a few differences in selection decisions made by the feature selection algorithms, so our next step is to immediately drop features that were eliminated both by RFE and Kepler. We will retain features that were slected by both, and decide how to handle the mixed results.

In [98]:
to_drop_classif = selection_results_classif[(selection_results_classif['Kepler Target 1 Results'] == 'Not Selected') 
                                                         & (selection_results_classif['RFE Target 1 Results'] == 'Not Selected')]
to_drop_classif

In [99]:
to_drop_reg = selection_results_reg[(selection_results_reg['Kepler Target 2 Results'] == 'Not Selected') 
                                                         & (selection_results_reg['RFE Target 2 Results'] == 'Not Selected')]
to_drop_reg

<i>It turns out from the above that not all features relevant for weather classification are relevant for percentage rain prediction</i>

In [100]:
train1 = selection_results_classif.drop(to_drop_classif.index)
train1

In [101]:
train2 = selection_results_reg.drop(to_drop_reg.index)
train2

**(d) Final Feature Selection by Intuition**

After the last series of steps, we see that the rest of the features we are left with have been either selected by both algorithms or by only one algorithm. At this point, we will now use our intuition to pick out the features we deem most relevant, based on our understanding of the data and the game so far.

Our argument is that, while the features retained have shown certain levels of importance based on our earlier steps, the ones relevant for our model are simply those that will cause definitive changes to the observed outcomes (weather, %rain) if they were changed. As such, we will retain features that: (i) capture information that a player could readily provide if asked, and (ii) show potential to change the outcomes if they were varied.

***The features we consider to match both criteria, supported by the preceding EDA, are:***
- Session Time
- Session Time Spent
- Track Length
- Track Temperature
- Air Temperature
- AI Difficulty
- Total Laps

These along with Time Offset, Weather and Rain Percentage will make up our final dataset. We are excluding session UID because we consider that its major role was to help us organise portions of the data according to their session group.

In [102]:
final_data_weather = afterdrop[[
    'M_SESSION_TIME',
    'M_SESSION_TIME_SPENT',
    'M_TRACK_LENGTH',
    'M_TRACK_TEMPERATURE',
    'M_AIR_TEMPERATURE',
    'M_AI_DIFFICULTY',
    'M_TOTAL_LAPS',
    'M_TIME_OFFSET',
    'M_WEATHER',
    'M_RAIN_PERCENTAGE'
    
]]

print(final_data_weather.shape)
final_data_weather.head()

Now that we have ended up with 8 input features, we will validate our choice using the following plots that show the ***variation of weather against each input feature grouped by time offsets***. This is the reverse of what we did earlier where we showed the distribution of each input feature with respect to weather and time offsets. We will repeat the same procedure for rain percentage as the dependent variable.

In [103]:
count = 1
colour = ['blue','darkred','darkblue']
plt.figure(figsize=(30,60))
for col in final_data_weather.columns:
    plt.subplot(5,2,count)
    final_data_weather.groupby(['M_TIME_OFFSET',col])['M_WEATHER'].mean().plot(lw=1,color=np.random.choice(colour))
    plt.ylabel('AVERAGE WEATHER') 
    plt.title('VARIATION OF WEATHER WITH {}'.format(col), fontsize = 16)
    plt.grid(False)
    if count<12:
        count+=1

In [104]:
count = 1
colour = ['blue','darkred','darkblue']
plt.figure(figsize=(30,60))
for col in final_data_weather.columns:
    plt.subplot(5,2,count)
    final_data_weather.groupby(['M_TIME_OFFSET',col])['M_RAIN_PERCENTAGE'].mean().plot(lw=1,color=np.random.choice(colour))
    plt.ylabel('AVERAGE RAIN PERCENTAGE') 
    plt.title('VARIATION OF RAIN PERCENTAGE WITH {}'.format(col), fontsize = 16)
    plt.grid(False)
    if count<12:
        count+=1

In [105]:
#final_data_weather.reset_index(drop=True, inplace=True)
final_data_weather.head()

In [106]:
final_data_weather.to_csv('./final_data_weather.csv', index = False)

## Part III: Modelling Methodology
<h4><b>Overview</b></h4>

This is the Machine Learning component of our solution to the FormulaAI Hack 2022 Competition. The workflow for this notebook is outlined as follows:
- Cross-Validation Setup
- Experiments with Feature Transformation
- Model Experimentations I: Weather Classification
- Classification Leaderboard Ranking
- Model Experimentations II: Regression
- Regression Leaderboard Ranking 
- Holdout Evaluation

In [107]:
import os

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None, 'display.max_rows', 100)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
#import deepchecks as dc

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

from xgboost import XGBClassifier
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRFRegressor
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.grid_search import HalvingGridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet

import random
import time
from datetime import datetime

***Read the data***

In [108]:
#final_data_weather = pd.read_csv('./final_data_weather.csv', index_col = False)
print(final_data_weather.shape)
final_data_weather.head(2)

In [109]:
weather_X = final_data_weather.drop('M_WEATHER', axis=1)
weather_y = final_data_weather['M_WEATHER']

rain_X = final_data_weather.drop('M_RAIN_PERCENTAGE', axis=1)
rain_y = final_data_weather['M_RAIN_PERCENTAGE']

print(weather_X.shape)
print(rain_X.shape)

<br>
<h4><b>1. Cross-Validation Set Up</b></h4>

***Creating train, test and validation sets***
    
We first split our data into train and test sets. The test set is our holdout set and will not be unlocked until the end of each of the 2 sequences of experiments for classification and regression, respectively. The validation set will be split out of the train data and will be used for primary evaluation and to compute cross validation scores in each of our experiments.

In [110]:
X_train, X_test, y_train, y_test = train_test_split(weather_X, weather_y, test_size=.25, random_state=42)

Considering the huge class imbalance in the weather target, ***we will implement repeated k-fold cross validation to further split our train data***. For our classification experiments, we will use ***repeated stratified k-fold cross validation***. We choose the value of 10 for *k* as this value has been shown empirically to yield test error rate estimates that suffer neither from excessively high bias nor from very high variance. In other words, we are choosing *k = 10* to achieve reasonable bias-variance trade-off during training.

In [111]:
#CV configuration for classificatoiclassification 
skfold = RepeatedStratifiedKFold(n_splits=10, n_repeats = 2, random_state=1)

#CV configuration for regression
kfold = RepeatedKFold(n_splits=10, n_repeats = 2, random_state=1)

Now, we will define a helper function that we will use for all our experiments. This function will also help to return validation scores for each experiment. We will customise this function a bit when we get to regression.

In [112]:
def training(X_train, y_train, model):

    fold_no = 1    
    n_scores, log_scores = [],[]
    for train_index, val_index in skfold.split(X_train, y_train):
        # select rows
        train_X, val_X = X_train.iloc[train_index], X_train.iloc[val_index]
        train_y, val_y = y_train.iloc[train_index], y_train.iloc[val_index]
        
        model.fit(train_X, train_y)
        
        n_scores.append(model.score(val_X, val_y))
        log_scores.append(log_loss(val_y, model.predict_proba(val_X), labels = [0,1,2]))
        print('For Fold {}, the Accuracy is {},'.format(str(fold_no), n_scores[fold_no - 1]), 
              'and the LogLoss is', log_scores[fold_no - 1])

        fold_no += 1
     
    #cv_score = cross_val_score(model, train_X, train_y, scoring='accuracy', cv=skfold, n_jobs=-1, error_score='raise')
    mean_accuracy, std_accuracy = np.mean(n_scores), np.std(n_scores)
    mean_loss, std_loss = np.mean(log_scores), np.std(log_scores)
    
    print('\n======================================')
    
    print('Mean Accuracy: %.3f (%.3f)' % (mean_accuracy, std_accuracy))
    #print('Mean CV Accuracy: %.3f (%.3f)' % (np.mean(cv_score), np.std(cv_score)))
    #
    #print('\n======================================')
    #
    print('Average LogLoss: {} ({})'.format(mean_loss, std_loss))

    return model, mean_loss

<br>
<h4><b>2. Experiments with Feature Transformation</b></h4>

For each algorithm we experiment with, we wil implement a variation using a scaled version of the training features. The following plots show the individual distribution of each input feature.

In [113]:
f, ax = plt.subplots(5,2, figsize = (30,60))
axx = ax.flatten()
for index, col in enumerate(final_data_weather.columns):
#     plt.figure(figsize=(15,8))
    sns.histplot(final_data_weather[col], ax=axx[index])

From the distribtion of the features, we see that it may be realistic to standardise the data given the absence of normality across features. We will run a variation of each of our modelling experiment with MinMaxScalar and narrow down the range of input values to [0, 10]. 

We wish to also test variations with RobustScaler to scale down input features in such a way that the influence of huge marginal points in the data are cancelled out.

***During these experiments with the scaled version of the data, we will make use of pipelines to avoid information leakage from our test data into the models we want to train.***

We define a pipeline construct below that implements the desired scaling on the training input features, then fits a model on this data. We will experiment with various models with and without feature transformation and see how they perform.

In [114]:
def scale_pipe(model_name, transformer, x_train, y_train):
    
    pipe = Pipeline([('scaler', transformer), ('model', model_name)])
    model_pipeline, mean_loss = training(x_train, y_train, pipe)
    
    return model_pipeline, mean_loss

<br>
<h4><b>3. Model Experimentations I: Weather Classification</b></h4>
Here are the 4 classification algorithms we will experiment with:

- XGBoost Classifier
- XGB Random Forest Classifier
- Light Gradient Boosted Machines Classifier
- Gradient Boosted Trees with CatBoost

##### **(a) XGBoost Classifier**

*Experiment 1: Without Standardisation*

In [115]:
xgb = XGBClassifier(eval_metric = 'mlogloss')
model_xgb, model_xgb_loss = training(X_train, y_train, xgb)
model_xgb_loss

Experiment 2: With Standardisation - MinMax and Robust

In [116]:
#MinMaxScaler
xgb2 = XGBClassifier(eval_metric = 'mlogloss')
transformer = MinMaxScaler(feature_range=(0, 10),copy=False)
xgb_scaled, xgb_scaled_loss = scale_pipe(xgb2, transformer, X_train, y_train)
xgb_scaled_loss

In [117]:
#RobustScaler
xgb3 = XGBClassifier(eval_metric = 'mlogloss')
transformer = RobustScaler()
xgb_scaled_2, xgb_scaled_2_loss = scale_pipe(xgb3, transformer, X_train, y_train)
xgb_scaled_2_loss

##### **(b) XGBoost Random Forest Classifier**
Experiment 1: Without Standardisation

In [118]:
xgbrf = XGBRFClassifier(eval_metric = 'mlogloss', n_estimators=10, subsample=0.9, colsample_bynode=0.2)
xgb_forest, xgb_forest_loss = training(X_train, y_train, xgbrf)
xgb_forest

Experiment 2: With Standardisation

In [119]:
#MinMaxScaler

xgbrf2 = XGBRFClassifier(eval_metric = 'mlogloss', n_estimators=20, subsample=0.9, colsample_bynode=0.2)

transformer = MinMaxScaler(feature_range=(0, 10),copy=False)
xgb_forest_scaled, xgb_forest_scaled_loss = scale_pipe(xgbrf2, transformer, X_train, y_train)

In [120]:
#RobustScaler

xgbrf3 = XGBRFClassifier(eval_metric = 'mlogloss', n_estimators=20, subsample=0.9, colsample_bynode=0.2)

transformer = RobustScaler()
xgb_forest_scaled_2, xgb_forest_scaled_2_loss = scale_pipe(xgbrf3, transformer, X_train, y_train)

##### **(c) Light Gradient Boosted Machines Classifier**

*First Experiment: Without Standardisation*

In [121]:
lgb = LGBMClassifier()
lgb_model, lgb_loss = training(X_train, y_train, lgb)

*Second Experiment: With Standardisation*

In [122]:
#MinMaxScaler

lgb2 = LGBMClassifier()
transformer = MinMaxScaler(feature_range=(0, 10),copy=False)
lgb_scaled, lgb_scaled_loss = scale_pipe(lgb2, transformer, X_train, y_train)

In [123]:
#RobustScaler

lgb3 = LGBMClassifier()
transformer = RobustScaler()
lgb_scaled_2, lgb_scaled_2_loss = scale_pipe(lgb3, transformer, X_train, y_train)

##### **(d) Gradient Boosted Trees with CatBoost Classifier**

*First Experiment: Without Standardisation*

In [124]:
ctb = CatBoostClassifier(verbose=0, n_estimators=100)
ctb_model, ctb_loss = training(X_train, y_train, ctb)

*Experiment 2: With Standardisation*

In [125]:
#MinMaxScaler

ctb2 = CatBoostClassifier(verbose=0, n_estimators=100)
transformer = MinMaxScaler(feature_range=(0, 10), copy=False)

ctb_scaled, ctb_scaled_loss = scale_pipe(ctb2, transformer, X_train, y_train)

In [126]:
#RobustScaler

ctb3 = CatBoostClassifier(verbose=0, n_estimators=100)
transformer = RobustScaler()

ctb_scaled_2, ctb_scaled_2_loss = scale_pipe(ctb3, transformer, X_train, y_train)

<br>
<h4><b>4. Classification Leaderboard</b></h4>

Before we unleash our holdout set which is 25% of our entire dataset, we will create a leaderboard of models built so far, to rank them by their mean logloss during cross-validation.

In [127]:
unpiped_models = [model_xgb_loss, xgb_forest_loss, lgb_loss, ctb_loss]
piped_models = [xgb_scaled_loss, xgb_forest_scaled_loss, lgb_scaled_loss, ctb_scaled_loss]
piped_models_2 = [xgb_scaled_2_loss, xgb_forest_scaled_2_loss, lgb_scaled_2_loss, ctb_scaled_2_loss]
model_index = ['XGBoost','XGBoost Random Forest','LightGBM', 'CatBoost Classifier']

pre_leaderboard_1 = pd.DataFrame({'Log Loss (Without Standardisation)': unpiped_models, 
                                  'Log Loss (With MinMaxScaler)': piped_models,
                                  'Log Loss (With RobustScaler)': piped_models_2}, index = model_index)

#pd.set_option('display.float_format', lambda x: '%.15f' % x)
pre_leaderboard_1

Now, let's set the tabe in ascending order, starting with the first column. We are ranking in ascending order as the goal of the learning algorithms is to minimise the log loss.

In [128]:
pre_leaderboard_1.nsmallest(4, 'Log Loss (Without Standardisation)')

In [129]:
pre_leaderboard_1.nsmallest(4, 'Log Loss (With MinMaxScaler)')

In [130]:
pre_leaderboard_1.nsmallest(4, 'Log Loss (With RobustScaler)')

***Key insights from the Classification Leaderboard:***

- XGBoost performed nearly the same with and without standardisation. However, it performed the best with RobustScaler.
- CatBoost performed the same  with and without standardisation
- XGBoost Random Forest performed better with standardisation, in particular with MinMax.
- In all cases, XGBoost outperformed other models by a large margin. 
- In all cases, XGB Random Forest came behind other models by a large order.

We will therefore select XGBoost with RobustScaler when evaluating with our test (holdout) set.

<br>
<h4><b>5. Model Experimentations II: Regression</b></h4>
Here are the 4 regression algorithms we will experiment with:

- RandomForest Regressor
- Light Gradient Boosted Trees Regressor (with Early Stopping)
- ElasticNet Regularised Regression with Grid Search Optimisation
- XGBoost Regressor

Before we proceed, we specify a new train-test split using the rain percentage data. We also change our k-fold algorithm to RepeatedKFold. In each of these experiments, we will use cross validation to give us insights into which model performs the best before we then unlock holdout to apply the best algorithm.

In [131]:
X1_train, X1_test, y1_train, y1_test = train_test_split(rain_X, rain_y, test_size=.25, random_state=42)
kfold = RepeatedKFold(n_splits=10, n_repeats = 2, random_state=1)

In [132]:
def reg_train(X1_train, y1_train, model):

    fold_no = 1    
    n_scores, mae_scores = [],[]
    for train_index, val_index in kfold.split(X1_train, y1_train):
        train_X, val_X = X1_train.iloc[train_index], X1_train.iloc[val_index]
        train_y, val_y = y1_train.iloc[train_index], y1_train.iloc[val_index]
        
        model.fit(train_X, train_y)
        
        n_scores.append(model.score(val_X, val_y))
        mae_scores.append(mean_absolute_error(val_y, model.predict(val_X)))
        print('For Fold {}, the Accuracy is {},'.format(str(fold_no), n_scores[fold_no - 1]), 
              'and the LogLoss is', mae_scores[fold_no - 1])
        
        fold_no += 1
     
    mean_accuracy, std_accuracy = np.mean(n_scores), np.std(n_scores)
    mean_mae, std_mae = np.mean(mae_scores), np.std(mae_scores)
    
    print('\n======================================')
    
    print('Average Accuracy Score: {} ({})'.format(mean_accuracy, std_accuracy))
    print('Average MAE and Standard Deviation: {} ({})'.format(mean_mae, std_mae))

    return model, mean_mae

***We also define a new pipeline construct that incorporates the cross-validation process.***

In [133]:
def reg_pipe(model_name, transformer, x_train, y_train):
    
    pipe = Pipeline([('scaler', transformer), ('cv', model_name)])
    scores = cross_val_score(pipe, x_train, y_train, scoring='neg_mean_absolute_error', 
                            cv=kfold, n_jobs=-1, error_score='raise')
    
    scaled_mae = np.mean(scores)
    
    return pipe, scaled_mae

##### **(a) Random Forest Regressor**

*First Experiment: Without Standardisation*

In [134]:
rforest = RandomForestRegressor(n_estimators=20)
rforest, rforest_mae = reg_train(X1_train, y1_train, rforest)

*Second Experiment: With Standardisation*

In [135]:
r_forest_scaled, r_forest_scaled_mae = reg_pipe(RandomForestRegressor(n_estimators = 20), MinMaxScaler(feature_range=(0,10)), X1_train, y1_train)
r_forest_scaled_mae

In [136]:
r_forest_scaled_2, r_forest_scaled_2_mae = reg_pipe(RandomForestRegressor(n_estimators = 20), RobustScaler(), X1_train, y1_train)
r_forest_scaled_2_mae

##### **(b) Light Gradient Boosted Trees Regressor**

*First Experiment: Without Standardisation (and with Early Stopping)*

In [137]:
lgb = LGBMRegressor()
lgbr, lgb_mae = reg_train(X1_train, y1_train, lgb)

*Second Experiment: With Standardisation (and Without Early Stopping)*

In [138]:
lgb_scaled, lgb_scaled_mae = reg_pipe(LGBMRegressor(), MinMaxScaler(feature_range=(0,10)), X1_train, y1_train)
lgb_scaled_mae

In [139]:
lgb_scaled_2, lgb_scaled_2_mae = reg_pipe(LGBMRegressor(), RobustScaler(), X1_train, y1_train)
lgb_scaled_2_mae

##### **(c) ElasticNet Regularised Regression with GridSearch Optimisation**

We will implement this algorithm by tuning both the ***L1 and L2*** penalties during training. A grid search is performed to find the ***alpha*** hyperparameter value that assigns the best weights to the contributions of the *L1* and *L2* penalties. The emerging alpha and L1 ratio are then used to fit the ElasticNet model.

In [140]:
ratios = np.arange(0, 1, 0.01)
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]

cv = RepeatedKFold(n_splits = 5, n_repeats=1, random_state=1)

start_model = ElasticNetCV(l1_ratio = ratios, alphas = alphas, cv = cv, n_jobs = -1)
# fit model
start_model.fit(X1_train, y1_train)

In [141]:
print(start_model.alpha_)
print(start_model.l1_ratio_)

In [142]:
#Implement cross-validation for ElasticNet across train data

enet = ElasticNet(alpha = start_model.alpha_,
                  l1_ratio = start_model.l1_ratio_)

enet_model, enet_mae = reg_train(X1_train, y1_train, enet)

In [151]:
enet_scaled, enet_scaled_mae = reg_pipe(enet, MinMaxScaler(feature_range=(0,10)), X1_train, y1_train)
enet_scaled_mae

In [152]:
enet_scaled_2, enet_scaled_2_mae = reg_pipe(enet, RobustScaler(), X1_train, y1_train)
enet_scaled_2_mae

##### **(d) XGBoost Regressor**

*First Experiment: Without Standardisation*

In [143]:
xgb_reg = XGBRegressor(objective='reg:squarederror')
xgbr, xgbr_mae = reg_train(X1_train, y1_train, xgb_reg)

*Second Experiment: With Standardisation*

In [144]:
#MinMaxScaler
xgbr1 = XGBRegressor(objective='reg:squarederror')
xgbr_scaled, xgb1_scaled_mae = reg_pipe(xgbr1, MinMaxScaler(feature_range=(0,10)), X1_train, y1_train)
xgb1_scaled_mae

In [145]:
#RobustScaler
xgbr2 = XGBRegressor(objective='reg:squarederror')
xgbr2_scaled, xgb2_scaled_mae = reg_pipe(xgbr2, RobustScaler(), X1_train, y1_train)
xgb2_scaled_mae

<br>
<h4><b>6. Regression Leaderboard</b></h4>
To rank our models properly, we take the absolute value of the negative MAE scores since the innacuracy of predictions is based upon the magnitude of the MAE loss and not the direction.

In [153]:
unpiped_models_reg = [rforest_mae, lgb_mae, enet_mae, xgbr_mae]
piped_models_reg = np.absolute([r_forest_scaled_mae, lgb_scaled_mae, enet_scaled_mae, xgb1_scaled_mae])
piped_models_reg2 = np.absolute([r_forest_scaled_2_mae, lgb_scaled_2_mae, enet_scaled_2_mae, xgb2_scaled_mae])

reg_model_index = ['Random Forest Regressor','LightGBM Regressor', 'ENet Regularised','XGBoost Regressor']
pre_leaderboard_2 = pd.DataFrame({'MAE (Without Standardisation)': unpiped_models_reg, 
                                  'MAE (With MinMaxScaler)': piped_models_reg,
                                  'MAE (With RobustScaler)': piped_models_reg2}, index = reg_model_index)

#pd.set_option('display.float_format', lambda x: '%.15f' % x)
pre_leaderboard_2

In [154]:
pre_leaderboard_2.nsmallest(4, 'MAE (Without Standardisation)')

In [155]:
pre_leaderboard_2.nsmallest(4, 'MAE (With MinMaxScaler)')

In [156]:
pre_leaderboard_2.nsmallest(4, 'MAE (With RobustScaler)')

From the leaderboard, we see that the champion model for the regression task is the XGBoost Regressor which outperformed others across the three feature variations. We are now going to unlock our test (holdout) set for evaluation using XGBoost with RobustScaler.

<h4><b>6. Holdout Evaluation</b></h4>

<b> (a) Top Performing Models </b>
- From the classification experiments, the champion model was the **XGBoost Classifier with RobustScaler preprocessing**
- From the regression experiments, the champion model was the **XGBoost Regressor with RobustScaler preprocessing**

<b> (b) Evaluating Model Accuracy </b>

Now, we unlock our holdout set and implement a pipeline to carry out the standardisation, prediction, and evaluation procedures. We will use the same evaluation metrics we used during cross-validation, namely: LogLoss Categorical Accuracy and Mean Absolute Error (MAE).

In [157]:
def class_pipe_score(estimator):
    
    '''
    This function constructs a pipeline of standardisation and modelling 
    steps for our classification champion model and returns the specified accuracy score 
    from the evaluation of the test set.
    '''
    
    pipe = Pipeline([('scaler', RobustScaler()), ('model', estimator)])
    model = pipe.fit(X_train, y_train)
    
    prediction = model.predict_proba(X_test)
    evaluation = log_loss(y_test, prediction, labels = [0,1,2])
    
    return model, evaluation

def reg_pipe_score(estimator):
    
    '''
    This function constructs a pipeline of standardisation and modelling 
    steps for our regression champion model and returns the specified accuracy score 
    from the evaluation of the test set.
    '''
    
    pipe = Pipeline([('scaler', RobustScaler()), ('model', estimator)])
    model = pipe.fit(X1_train, y1_train)
    
    prediction = model.predict(X1_test)
    evaluation = mean_absolute_error(y1_test, prediction)
    
    return model, evaluation

In [158]:
class_model, class_eval = class_pipe_score(xgb_scaled_2)

In [159]:
reg_model, reg_eval = reg_pipe_score(xgbr2_scaled)

In [160]:
print(class_eval)
print(reg_eval)

***Here's a summary table of the two champion models we obtained and their accuracy scores.***

In [161]:
metrics = ['LogLoss','MAE']
scores = [class_eval,reg_eval]
index = ['XGBoost Classifier','XGBoost Regressor']

#pd.set_option('display.float_format', lambda x: '%.15f' % x)
model_summary = pd.DataFrame({'Evaluation Metric':metrics, 'Evaluation Score':scores}, index = index)
model_summary

<h4> </h4>

## Part IV: Predictions and Exporting

In [168]:
import pickle
import json
pd.reset_option('display.precision')

In [183]:
def twin_predictor(input_data, estimators):
    """
    reads data, processes it for the classification and regression tasks, 
    passes it into "models" (a stack of the classifier and regressor) for inference,
    and returns a json response of predictions across the time intervals
    of {5,10,15,30,60} minutes after a session timestamp.
    """
    
    #Split data into weather data and rain data to be taken in by the respective models
    weather_input = input_data.drop('M_WEATHER', axis = 1)
    rain_input = input_data.drop('M_RAIN_PERCENTAGE', axis = 1)
    X_list = [weather_input, rain_input] 

    export = dict()
    intervals = [5,10,15,30,60]
    weather_type = ['Clear', 'Light Cloud', 'Overcast', 'Light Rain', 'Heavy Rain', 'Storm']
    
    for time in intervals:
        
        #Assign selected offset to the offset fields
        weather_input['M_TIME_OFFSET'] = time
        rain_input['M_TIME_OFFSET'] = time
        
        #Run inference
        prediction1 = estimators[0].predict(X_list[0])
        prediction2 = estimators[1].predict(X_list[1])
        
        #Output results
        for i,j in enumerate(weather_type):
            if int(prediction1) == i:
                export['{} min'.format(time)] = {
                    'Weather Type': '{} ({})'.format(i,j),
                    'Rain Percentage' : np.format_float_positional(prediction2[0], 2)
                }
        
    return json.dumps(export)
    

In [194]:
full_test_data = final_data_weather[(final_data_weather.drop('M_WEATHER', axis = 1).isin(X_test)).all(axis=1)]
test_data = full_test_data.iloc[2000:2001]

In [195]:
models = [class_model, reg_model]
twin_predictor(test_data, models)

**(b) Exporting, loading and testing**

In [196]:
twin_ai_features = './twin_ai_features.pkl'
pickle.dump(final_data_weather.columns, open(twin_ai_features, 'wb'))

estimators = [class_model, reg_model]
twin_ai_predictor = './estimators.pkl'
pickle.dump(estimators, open('twin_ai_predictor', 'wb'))

In [197]:
load_model = pickle.load(open('twin_ai_predictor', 'rb'))
make_prediction = twin_predictor(test_data, load_model)

print(make_prediction)

## Next Steps

Having completed the machine learning workflow for this challenge, we will continue work on the following objectives to take our models out of the notebook and into the real world:
- Deploy the models into a containerised service on Kubernetes via Oracle Cloud
- Set up real-time inference pipeline for online predictions
- Expose the inference API to be consumed by a frontend UI built with React.js and fastAPI.
- Set up MLOps infrastructure and workflows to monitor model performance in production
- Continuously retraining model to achieve better results and to ensure stability of model quality in production

## Closing Remarks/Acknowledgements

We sincerely thank Hackmakers and Oracle for hosting this amazing contest. It was a fun and challenging experience, albeit with a mix of severe external limitations we had to get our way around. We look forward to seeing the winning solutions and how others approached the problem. We hope to continue to learn more about the Formula One ecosystem, and discover more of the amazing work that the RedBull Racing Research team is doing.

 #### **Notebook by Team Twin AI**
 
 <br><br>